In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Concatenate
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Input
from keras import backend as K
from keras.models import load_model
import pickle
import pandas as pd
import re
import numpy as np
import random
from keras.utils.vis_utils import plot_model
import keras.callbacks
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
from numpy.random import seed; seed(123)
from tensorflow import set_random_seed; set_random_seed(123)
from sklearn.metrics import roc_auc_score
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.ensemble import RandomForestClassifier

In [ ]:
##### test flowSOM  #####
processed_data = pickle.load( open( "result_04_processed_data_no_scale.obj", "rb" ) )
cytof_files = processed_data["cytof_files"]
expr_list = processed_data["expr_list"]

r1 = [t1==t1 for t1 in cytof_files.CMV_Ab] 
cytof_files = cytof_files.loc[r1,:]
expr_list = expr_list[r1]
print(expr_list.shape)

gating_result = pd.read_csv("result_07_summary_statistics.csv")

dat = pd.merge(cytof_files, gating_result, how='inner', left_on="name",right_on=["fcs_files"])
coln = gating_result.columns[1:]
y = dat.CMV_Ab.values> 2
x = dat[coln].values
x = np.nan_to_num(x)
print(x.shape);print(y.shape)

##### split train, validation and test######
train_id = [i for i in range(len(x)) if dat.study_accession.iloc[i] not in ["SDY515","SDY519"]]
valid_id = [i for i in range(len(x)) if dat.study_accession.iloc[i]=="SDY515"]
test_id = [i for i in range(len(x)) if dat.study_accession.iloc[i]=="SDY519"]

x_train = x[train_id]
x_valid = x[valid_id]
x_test = x[test_id]

y_train = y[train_id]
y_valid = y[valid_id]
y_test = y[test_id]

clf = RandomForestClassifier(n_estimators=1000)
clf.fit(x_train, y_train)


y_true = y_test
y_scores = clf.predict_proba(x_test)[:,1]
#print(y_scores)
print(roc_auc_score(y_true, y_scores))

with open("result_08_flowSOM_RF_ROC.obj", "wb") as f:
    pickle.dump({"true":y_test,"score":y_scores}, f)

In [ ]:
##### load data #####
processed_data = pickle.load( open( "result_04_processed_data_no_scale.obj", "rb" ) )
cytof_files = processed_data["cytof_files"]
expr_list = processed_data["expr_list"]

r1 = [t1==t1 for t1 in cytof_files.CMV_Ab] 
cytof_files = cytof_files.loc[r1,:]
expr_list = expr_list[r1]
print(expr_list.shape)

gating_result = pd.read_csv("result_06_summary_statistics.csv")

dat = pd.merge(cytof_files, gating_result, how='inner', left_on="name",right_on=["fcs_files"])
coln = gating_result.columns[1:]
y = dat.CMV_Ab.values> 2
x = dat[coln].values
x = np.nan_to_num(x)
print(x.shape);print(y.shape)

##### split train, validation and test######
train_id = [i for i in range(len(x)) if dat.study_accession.iloc[i] not in ["SDY515","SDY519"]]
valid_id = [i for i in range(len(x)) if dat.study_accession.iloc[i]=="SDY515"]
test_id = [i for i in range(len(x)) if dat.study_accession.iloc[i]=="SDY519"]

x_train = x[train_id]
x_valid = x[valid_id]
x_test = x[test_id]

y_train = y[train_id]
y_valid = y[valid_id]
y_test = y[test_id]

clf = RandomForestClassifier(n_estimators=1000)
clf.fit(x_train, y_train)


y_true = y_test
y_scores = clf.predict_proba(x_test)[:,1]
#print(y_scores)
print(roc_auc_score(y_true, y_scores))

with open("result_08_10k_RF_ROC.obj", "wb") as f:
    pickle.dump({"true":y_test,"score":y_scores}, f)